In [71]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
import csv
import numpy as np
import os
import sqlite3
import copy

print(f"Python Version = {sys.version}")
print(f"Pandas Version = {pd.__version__}")

Python Version = 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
Pandas Version = 0.25.1


# Definitions

1) Fine Fuel Moisture Code
The Fine Fuel Moisture Code (FFMC) is a numeric rating of the moisture content of litter and other cured fine fuels. This code is an indicator of the relative ease of ignition and the flammability of fine fuel.

2) Duff Moisture Code
The Duff Moisture Code (DMC) is a numeric rating of the average moisture content of loosely compacted organic layers of moderate depth. This code gives an indication of fuel consumption in moderate duff layers and medium-size woody material.

3) Drought Code
The Drought Code (DC) is a numeric rating of the average moisture content of deep, compact organic layers. This code is a useful indicator of seasonal drought effects on forest fuels and the amount of smoldering in deep duff layers and large logs.

4)  Initial Spread Index
The Initial Spread Index (ISI) is a numeric rating of the expected rate of fire spread. It combines the effects of wind and the FFMC on rate of spread without the influence of variable quantities of fuel.

https://cwfis.cfs.nrcan.gc.ca/background/summary/fwi

5) Temp
Temperature in Celsius degrees

6) RH
Relative humidity in %

7) Wind
Wind speed in km/h

8) Rain
Outside rain in mm/m2

9) Area
The burned area of the forest (in ha), ha = hectare = 2.47 acres

10) X
x-axis spatial coordinate within the Montesinho park map: 1 to 9 2

11) Y
y-axis spatial coordinate within the Montesinho park map: 2 to 9 3

Literature:

https://news.uci.edu/2019/09/17/uci-team-uses-machine-learning-to-help-tell-which-wildfires-will-burn-out-of-control/

In the original paper (Portugal - Montesinho fires), the best result (predicting burned area) was only using the 4 weather conditions: RH/Wind/Rain/Temp, and an SVM.

This: https://datasetsearch.research.google.com/search?query=wildfire%20area&docid=fchFFzqh0O3qjX3PAAAAAA%3D%3D has georeferenced and date/time referenced with areas. Could combine with weather data for those dates/times.

# Query shape of datasets and feature names

In [8]:
os.getcwd()

'C:\\DDRIVE\\W210 Capstone\\wildfireplus\\EDA'

## Montesinho

In [21]:
montesinho = pd.read_csv('../data/Montesinho/forestfires.csv')

In [22]:
print(montesinho.shape)
montesinho.head()

(517, 13)


,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


## US Fires 1.88 Million

In [63]:
db = sqlite3.connect('../data/188millionuswildfires/FPA_FOD_20170508.sqlite')

In [ ]:
cursorobj = db.cursor()
cursorobj.execute('SELECT name from sqlite_master where type = "table"')
tables1 = cursorobj.fetchall()

In [ ]:
tables2 = copy.deepcopy(tables1)
del tables2[18:21]
tables2
del tables2[19]
tables2

In [80]:
for table in tables2:
    query = "SELECT * FROM " + table[0]
    #print(query)
    cursorobj.execute(query)
    names = [description[0] for description in cursorobj.description]
    print(table, names)

('spatial_ref_sys',) ['srid', 'auth_name', 'auth_srid', 'ref_sys_name', 'proj4text', 'srtext']
('spatialite_history',) ['event_id', 'table_name', 'geometry_column', 'event', 'timestamp', 'ver_sqlite', 'ver_splite']
('sqlite_sequence',) ['name', 'seq']
('geometry_columns',) ['f_table_name', 'f_geometry_column', 'geometry_type', 'coord_dimension', 'srid', 'spatial_index_enabled']
('spatial_ref_sys_aux',) ['srid', 'is_geographic', 'has_flipped_axes', 'spheroid', 'prime_meridian', 'datum', 'projection', 'unit', 'axis_1_name', 'axis_1_orientation', 'axis_2_name', 'axis_2_orientation']
('views_geometry_columns',) ['view_name', 'view_geometry', 'view_rowid', 'f_table_name', 'f_geometry_column', 'read_only']
('virts_geometry_columns',) ['virt_name', 'virt_geometry', 'geometry_type', 'coord_dimension', 'srid']
('geometry_columns_statistics',) ['f_table_name', 'f_geometry_column', 'last_verified', 'row_count', 'extent_min_x', 'extent_min_y', 'extent_max_x', 'extent_max_y']
('views_geometry_colum

In [93]:
usfires2 = pd.read_sql_query("SELECT datetime(DISCOVERY_DATE) as DISCOVERY_DATE, \
                             DISCOVERY_TIME, FIRE_SIZE, LATITUDE, LONGITUDE, \
                             datetime(CONT_DATE) as CONT_DATE, CONT_TIME \
                             FROM Fires",db)

In [94]:
usfires2.shape

(1880465, 7)

In [95]:
usfires2.head()

,DISCOVERY_DATE,DISCOVERY_TIME,FIRE_SIZE,LATITUDE,LONGITUDE,CONT_DATE,CONT_TIME
0,2005-02-02 00:00:00,1300,0.10,40.036944,-121.005833,2005-02-02 00:00:00,1730
1,2004-05-12 00:00:00,0845,0.25,38.933056,-120.404444,2004-05-12 00:00:00,1530
2,2004-05-31 00:00:00,1921,0.10,38.984167,-120.735556,2004-05-31 00:00:00,2024
3,2004-06-28 00:00:00,1600,0.10,38.559167,-119.913333,2004-07-03 00:00:00,1400
4,2004-06-28 00:00:00,1600,0.10,38.559167,-119.933056,2004-07-03 00:00:00,1200
